In [827]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import scipy
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.preprocessing import MinMaxScaler

In [828]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [829]:
# Hyper parameters
num_classes = 5
num_epochs = 300
batch_size = 64
# batch_size = 64
learning_rate = 0.01

sequence_length = 1000 # 1セグメントあたりのサンプル数 250Hz * 5s
input_size = 1 # 特徴量の種類？
hidden_size = 120 # 論文でいうhidden layerはこれ?
num_layers = 2 # Two-layered lstm?

In [830]:
!pwd
!ls

/mnt/c/Users/grpro/workspace/grad_thesis/lstm_cnn_ensemble
CNN.ipynb   LSTM_colab.ipynb	 README.md  env
LSTM.ipynb  LSTM_tutorial.ipynb  data	    output.png


In [831]:
default_sample_rate = 2000
down_sample_rate = 250
ratio = default_sample_rate // down_sample_rate

In [832]:
# num_classes人分のdataframeを作る
radar_frame_list = []
scaler = MinMaxScaler((-1, 1)) # Min-Max Scaler -1~1

for i in range(1, num_classes + 1):
    wave_dem_2d = []
    file_path = "./data/radar_%02d.csv" % i
    radar_frame = pd.read_csv(file_path)
    wave = radar_frame.to_numpy().flatten()
    wave_dem = signal.decimate(wave, ratio)
    for i in range(len(wave_dem)):
      wave_dem_2d.append([wave_dem[i]])
    # wave_dem_ts = torch.from_numpy(wave_dem.astype(np.float32)).clone() # tensorにする
    print(i, len(wave), len(wave_dem_2d) // 900)

    remaining = len(wave_dem_2d)
    n = 0
    n_stop = sequence_length
    wave_segments = []

    while n_stop < len(wave_dem_2d):
        n_start = 0 + ((sequence_length - 1) - (350 - 1)) * n
        n_stop = n_start + sequence_length
        tmp = []
        wave_segments.append(scaler.fit_transform(wave_dem_2d[n_start:n_stop]))
        n += 1
    
    radar_frame_list.append(wave_segments)

151899 1215199 168
155587 1244699 172
150337 1202699 167
150768 1206149 167
152524 1220199 169


In [833]:
radar_frame_list[0][0]

array([[-0.4311903 ],
       [-0.397682  ],
       [-0.36072063],
       [-0.32017558],
       [-0.27964272],
       [-0.23889709],
       [-0.19975085],
       [-0.16227468],
       [-0.1265833 ],
       [-0.09293633],
       [-0.06044722],
       [-0.02922139],
       [ 0.00151319],
       [ 0.03188227],
       [ 0.06203263],
       [ 0.09204485],
       [ 0.1214989 ],
       [ 0.15016363],
       [ 0.17738266],
       [ 0.20264883],
       [ 0.22540391],
       [ 0.24515136],
       [ 0.26163516],
       [ 0.27466233],
       [ 0.28434434],
       [ 0.29091749],
       [ 0.29482184],
       [ 0.29663487],
       [ 0.2969817 ],
       [ 0.29654303],
       [ 0.29592123],
       [ 0.29565188],
       [ 0.29613066],
       [ 0.29760649],
       [ 0.30018929],
       [ 0.30384166],
       [ 0.30842755],
       [ 0.31371988],
       [ 0.31944587],
       [ 0.32531512],
       [ 0.33105008],
       [ 0.33642158],
       [ 0.34126118],
       [ 0.34547798],
       [ 0.34905185],
       [ 0

In [834]:
type(wave_dem_2d) # tensorにはしていない

list

In [835]:
wave_dem[0]

-2.3435893667948337e-05

In [836]:
print(radar_frame.to_numpy())

[[-2.37119380e-05]
 [-2.38183832e-05]
 [-2.39258871e-05]
 ...
 [-7.11852346e-06]
 [-6.73462954e-06]
 [-6.34069284e-06]]


In [837]:
labels = []
for i in range(len(radar_frame_list)):
    for j in range(len(radar_frame_list[i])):
        labels.append(i + 1)

labels_df = pd.Series(labels)
torch.tensor(labels_df)

tensor([1, 1, 1,  ..., 5, 5, 5])

In [838]:
df = pd.DataFrame(radar_frame_list)
tmp = df.to_numpy().flatten()
df = pd.Series(tmp).dropna()
tmp = df.to_numpy().flatten()

tmp_labels = labels_df.to_numpy().flatten()
for i in reversed(range(len(tmp))):
  if len(tmp[i]) != sequence_length:
    print(i, len(tmp[i]))
    tmp = np.delete(tmp, i)
    tmp_labels = np.delete(tmp_labels, i)

df = pd.Series(tmp)
labels_df = pd.Series(tmp_labels)

1170 425
935 619
703 838
472 888
233 450


In [839]:
df, type(df), type(df[0]), type(df[0][0]), type(df[0][0][0]), len(df[0])

(0       [[-0.43119030027867716], [-0.39768199524563536...
 1       [[-0.8735282292357403], [-0.8559558009192504],...
 2       [[-0.6323235077932776], [-0.6233832414220843],...
 3       [[-0.7800916387927728], [-0.7991681872044234],...
 4       [[0.17223992735245988], [0.19803851588219734],...
                               ...                        
 1161    [[0.35360439006574296], [0.37555350233418683],...
 1162    [[0.14708770411253336], [0.1446543093132371], ...
 1163    [[-0.1086875547518791], [-0.10866502525283459]...
 1164    [[0.9191947403695329], [0.9420051599104073], [...
 1165    [[0.512804350202705], [0.5158244517411077], [0...
 Length: 1166, dtype: object,
 pandas.core.series.Series,
 numpy.ndarray,
 numpy.ndarray,
 numpy.float64,
 1000)

In [840]:
labels_df

0       1
1       1
2       1
3       1
4       1
       ..
1161    5
1162    5
1163    5
1164    5
1165    5
Length: 1166, dtype: int64

In [841]:
class MyDataset(Dataset):
    def __init__(self, dataset, labels, root_dir, transform=None) -> None:
        # super().__init__()
        self.radar_heartbeat = dataset
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform

    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
          idx = idx.tolist()
        # heartbeat_segment = torch.FloatTensor(self.radar_heartbeat[idx])
        # subject_label = torch.LongTensor(self.labels[idx])
        # if self.transform: # transform適用されず?
        # heartbeat_segment = self.transform(self.radar_heartbeat[idx]) # tensorに
        onehot_label = torch.eye(num_classes)[self.labels[idx] - 1] # one hot encording
        # onehot_label = self.transform(self.labels[idx]) # tensorに

        # return heartbeat_segment, subject_label
        # return torch.from_numpy(self.radar_heartbeat.to_numpy().copy()), torch.tensor(labels_df)    
        # return torch.stack(self.radar_heartbeat.to_numpy().tolist()), torch.tensor(labels_df)    
        # return self.radar_heartbeat[idx], labels_df[idx]
        return torch.tensor(self.radar_heartbeat[idx]), onehot_label



    def __len__(self):
        return len(self.radar_heartbeat)

In [842]:
dataset = MyDataset(df, labels_df, "./data/", transform=transforms.ToTensor())

In [843]:
seg, lab = next(iter(dataset))

In [844]:
type(seg), len(seg), lab

(torch.Tensor, 1000, tensor([1., 0., 0., 0., 0.]))

In [845]:
train_size = int(0.8 * len(df.values))
test_size = len(df.values) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])
print(f"full: {len(dataset)} -> train: {len(train_set)}, test: {len(test_set)}")

full: 1166 -> train: 932, test: 234


In [846]:
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

In [847]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes) -> None:
    super().__init__()
    self.num_layers = num_layers
    self.hidden_size = hidden_size
    # self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    # print(x.size(), h0.size(), c0.size())
    # out, _ = self.rnn(x, h0)
    out, _ = self.lstm(x, (h0, c0))

    out = out[:, -1, :]

    out = self.fc(out)

    return out

In [848]:
model = LSTM(input_size, hidden_size, num_layers, num_classes)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)

In [849]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (signals, labels) in enumerate(train_loader):
    signals = torch.tensor(signals)
    signals = signals.float()
    signals = signals.to(device)
    labels = labels.to(device)

    # print(signals.size())
    outputs = model(signals)
    # print(outputs)
    loss = criterion(outputs, labels) # will check the shapes of outputs and labels

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'Epoch [{epoch+1}/`{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

/tmp/ipykernel_511/13283608.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(signals)


Epoch [1/`300], Step [30/30], Loss: 1.7956
Epoch [2/`300], Step [30/30], Loss: 1.5694
Epoch [3/`300], Step [30/30], Loss: 1.4702
Epoch [4/`300], Step [30/30], Loss: 2.1177
Epoch [5/`300], Step [30/30], Loss: 1.6322
Epoch [6/`300], Step [30/30], Loss: 1.4918
Epoch [7/`300], Step [30/30], Loss: 1.5190
Epoch [8/`300], Step [30/30], Loss: 1.2990
Epoch [9/`300], Step [30/30], Loss: 1.4224
Epoch [10/`300], Step [30/30], Loss: 1.6376
Epoch [11/`300], Step [30/30], Loss: 1.8575
Epoch [12/`300], Step [30/30], Loss: 1.4125
Epoch [13/`300], Step [30/30], Loss: 1.5525
Epoch [14/`300], Step [30/30], Loss: 1.3632
Epoch [15/`300], Step [30/30], Loss: 1.5081
Epoch [16/`300], Step [30/30], Loss: 1.4916
Epoch [17/`300], Step [30/30], Loss: 1.6327
Epoch [18/`300], Step [30/30], Loss: 1.3919
Epoch [19/`300], Step [30/30], Loss: 1.7424
Epoch [20/`300], Step [30/30], Loss: 1.6839
Epoch [21/`300], Step [30/30], Loss: 1.5574
Epoch [22/`300], Step [30/30], Loss: 1.9009
Epoch [23/`300], Step [30/30], Loss: 1.41

In [850]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  softmax = nn.Softmax()
  for i, (signals, one_hot_labels) in enumerate(test_loader):
    signals = torch.tensor(signals)
    signals = signals.float()
    signals = signals.to(device)
    one_hot_labels = one_hot_labels.to(device)
    # print(len(one_hot_labels))
    outputs = model(signals)
    for j, out in enumerate(outputs):
      outputs[j] = softmax(out)

    _, predicted = torch.max(outputs.data, 1) # predicted per batch size

    n_samples += one_hot_labels.size(0) # add batch_size
    # labels = [] # labels per batch size
    for k, labels in enumerate(one_hot_labels):
      # labels.append(torch.argmax(one_hot_labels[k]))
      if predicted[k] == torch.argmax(labels):
        # print(predicted[k], torch.argmax(labels[k]))
        n_correct += 1

    acc = 100.0 * n_correct / n_samples
    print(f'{n_correct} / {n_samples} = Acc: {acc} %')
    # print(f'Accuracy of the network on the 10000 test images: {acc} %')

14 / 32 = Acc: 43.75 %
28 / 64 = Acc: 43.75 %
41 / 96 = Acc: 42.708333333333336 %
56 / 128 = Acc: 43.75 %
73 / 160 = Acc: 45.625 %
82 / 192 = Acc: 42.708333333333336 %
91 / 224 = Acc: 40.625 %
94 / 234 = Acc: 40.17094017094017 %


/tmp/ipykernel_511/3918647096.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  signals = torch.tensor(signals)
/tmp/ipykernel_511/3918647096.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  outputs[j] = softmax(out)


In [851]:
predicted.size()

torch.Size([10])

In [852]:
torch.argmax(labels)
labels.size()

torch.Size([5])

In [853]:
predicted, (labels)

(tensor([0, 0, 0, 2, 1, 0, 0, 0, 1, 1], device='cuda:0'),
 tensor([1., 0., 0., 0., 0.], device='cuda:0'))

In [854]:
result = (predicted == torch.argmax(labels[len(labels)-1]))
result, labels[len(labels)-1]

(tensor([ True,  True,  True, False, False,  True,  True,  True, False, False],
        device='cuda:0'),
 tensor(0., device='cuda:0'))